<a href="https://colab.research.google.com/github/Aya11ali/Shouf/blob/main/Video_Summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q moviepy faster-whisper langdetect
!pip install -q transformers einops accelerate langchain bitsandbytes sentencepiece langchain-community

In [ ]:
import subprocess
import os
from abc import ABC, abstractmethod
from faster_whisper import WhisperModel

from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

from langchain import HuggingFacePipeline, PromptTemplate, LLMChain
from transformers import AutoTokenizer, pipeline
import transformers
import torch
from concurrent.futures import ThreadPoolExecutor

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
logging.getLogger("huggingface_hub").setLevel(logging.ERROR)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
class AudioExtractionError(Exception):
    """Custom exception for audio extraction errors."""
    pass

In [ ]:
class IAudioExtraction(ABC):
  @abstractmethod
  def __init__(self,video_path:str , audio_path:str = 'extracted_audio.mp3'):
    pass

  @abstractmethod
  def check_video_path(self)-> None:
    pass

  @abstractmethod
  def extract_audio(self) -> None:
    pass

  @abstractmethod
  def check_audio_validation(self)-> bool:
    pass

class AudioExtraction(IAudioExtraction):
  def __init__(self,video_path:str , audio_path:str = 'extracted_audio.mp3'):
    self.video_path = video_path
    self.audio_path = audio_path
    self.result = None
    self.extract_audio()

  def check_video_path(self) -> None:
    if not os.path.exists(self.video_path):
      raise ValueError(f"Video path {self.video_path} does not exist")

  def extract_audio(self) -> None:
    self.check_video_path()
    command = [
          "ffmpeg",
          "-y",
          "-i", self.video_path,
          "-vn",
          "-acodec", "libmp3lame",
          self.audio_path
      ]

    self.result = subprocess.run(command, stdout=subprocess.PIPE,
                              stderr=subprocess.PIPE, text=True)
    self.check_audio_validation()


  def check_audio_validation(self) -> bool:
      if self.result.returncode != 0:
          raise AudioExtractionError(f"Failed to extract audio from video. Error: {self.result.stderr}")

      logging.info("Audio extracted successfully")
      return True

In [ ]:
class IAudio_Model_Loader (ABC):
  @abstractmethod
  def __init__ (self,model_size="medium",compute_type="int8"):
    pass

  @abstractmethod
  def load_model (self):
    pass

class Audio_Model_Loader (IAudio_Model_Loader):
  def __init__ (self,model_size="medium",compute_type="int8"):
    self.model_size = model_size
    self.compute_type = compute_type
    self.model = None
    self.load_model()

  def load_model (self):
    self.model = WhisperModel(self.model_size, compute_type=self.compute_type)

In [ ]:
class IAudio_Transcriber(ABC):
  @abstractmethod
  def __init__ (self,audio_path):
    pass

  @abstractmethod
  def transcribe_audio(self):
    pass

class Audio_Transcriber(IAudio_Transcriber):
  def __init__ (self,audio_path,model):
    self.audio_path = audio_path
    self.segments = None
    self.model = model
    self.transcription=""
    self.transcribe_audio()

  def transcribe_audio(self):
    self.segments, _ = self.model.transcribe(self.audio_path, beam_size=5)

    for segment in self.segments:
        self.transcription += segment.text.strip() + " "

    self.transcription = self.transcription.strip()

In [ ]:
DetectorFactory.seed = 0

class IDetectLanguage(ABC):
  @abstractmethod
  def detect_language(self,text:str)-> str:
    pass

class DetectLanguage(IDetectLanguage):
  def detect_language(self,text:str)-> str:
    try:

      return detect(text)
    except LangDetectException:
      return "Unknown"

In [ ]:
class VideoToTextController(ABC):
  def __init__ (self, video_path : str , audio_path : str = 'extracted_audio.mp3'):
    self.video_path = video_path
    self.audio_path = audio_path

    self.audio_extraction = AudioExtraction(self.video_path,self.audio_path)
    self.audio_model_loader = Audio_Model_Loader()

    self.audio_transcriber = Audio_Transcriber(self.audio_path,self.audio_model_loader.model)

In [ ]:
if __name__ == "__main__":
  videoText = VideoToTextController("/content/22.mp4")
  print(videoText.audio_transcriber.transcription)

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

In [ ]:
class ISummarizer(ABC):
    @abstractmethod
    def __init__(self, text: str):
        pass

    @abstractmethod
    def summarize_text(self) -> str:
        pass


class Summarizer(ISummarizer):
    def __init__(self, text: str, model: str = "meta-llama/Llama-2-7b-chat-hf"):
        self.text = text
        self.model = model

        try:
            self.tokenizer = AutoTokenizer.from_pretrained(model)
            self.pipeline = pipeline(
                "text-generation",
                model=self.model,
                tokenizer=self.tokenizer,
                torch_dtype=torch.float16,
                device=0,
                trust_remote_code=True,
                max_length=512,
                do_sample=True,
                top_k=10,
                num_return_sequences=1,
                eos_token_id=self.tokenizer.eos_token_id
            )

            self.template = """
                              You are a helpful summarization assistant.

                              Write a clear and concise **paragraph summary** of the following text, delimited by triple backticks.
                              The summary should capture the **main ideas** and **important details** without exceeding **300 words**.

                              ```{text}```

                              PARAGRAPH SUMMARY (max 300 words):
                          """
            prompt = PromptTemplate(template=self.template, input_variables=["text"])
            llm = HuggingFacePipeline(pipeline=self.pipeline)
            self.llm_chain = LLMChain(prompt=prompt, llm=llm)
        except Exception as e:
            raise RuntimeError(f"Failed to initialize model or tokenizer: {e}")

    def summarize_text(self) -> str:
        try:
            # Use LLMChain to generate the summary
            summary = self.llm_chain.run({"text": self.text})
            return summary
        except Exception as e:
            raise RuntimeError(f"Failed to summarize text: {e}")
